# Test few different padas data frame as agent tool:

* Get data with `FinancialDataYahoo` class in `financial_data.py`
* Get data with `Data` class in price data. (Historical price data)
* Define `pandas_agent` functuion to create DataFrame agent.
* Set pandas_agent as tools
* Create other tools like searching and python REPL
* Set up Memory
* Initialize agent

In [1]:
from price_data import Data
from financial_data import FinancialDataFinnHub, FinancialDataYahoo
import os
import pandas as pd

finhub_api_key = os.environ.get("FINHUB_API_KEY")
symbol = "AMD"

In [2]:
price = Data(symbol, 1).preprocess_data_2[-100:]
price_direction = Data(symbol, 50).add_arrow
amd = FinancialDataFinnHub(symbol, finhub_api_key)
amd_y = FinancialDataYahoo(symbol)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
stats = amd_y.stats

C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(elt)
C:\Users\Masa\anaconda3\lib\site-packages\yahoo_fin\stock_info.py:336: F

In [4]:
# Last 3 years of balancesheet
annual_bs = pd.concat(amd.bs_list_a[:3])

## Set OpenAI Chat model

In [5]:
from langchain.chat_models import ChatOpenAI
import os
llm = ChatOpenAI(temperature=0, openai_api_key=os.environ.get("OPENAI_API_KEY"))

## Define Pandas Agents:

In [6]:
from langchain.agents import create_pandas_dataframe_agent

def pandas_agent(df, input=""):
    return create_pandas_dataframe_agent(llm, df, verbose=True, openai_api_key=os.environ.get("OPENAI_API_KEY"))

## Create pandas DataFrame Agents

In [7]:
stat_agent = pandas_agent(stats)
balance_sheet_agent = pandas_agent(annual_bs)
price_agent = pandas_agent(price)

In [8]:
price_agent.run("What is the last 10 days of price movement?")



> Entering new AgentExecutor chain...
Thought: We need to look at the last 10 rows of the 'Close' column.
Action: python_repl_ast
Action Input: df['Close'][-10:]
Observation: 190    89.870003
191    89.779999
192    89.940002
193    90.110001
194    88.430000
195    87.570000
196    83.800003
197    85.940002
198    87.440002
199    89.370003
Name: Close, dtype: float64
Thought:We have the last 10 days of closing prices.
Action: python_repl_ast
Action Input: df['Close'][-10:].tolist()
Observation: [89.87000274658203, 89.77999877929688, 89.94000244140625, 90.11000061035156, 88.43000030517578, 87.56999969482422, 83.80000305175781, 85.94000244140625, 87.44000244140625, 89.37000274658203]
Thought:We have the last 10 days of closing prices as a list.
Final Answer: [89.87000274658203, 89.77999877929688, 89.94000244140625, 90.11000061035156, 88.43000030517578, 87.56999969482422, 83.80000305175781, 85.94000244140625, 87.44000244140625, 89.37000274658203]

> Finished chain.


'[89.87000274658203, 89.77999877929688, 89.94000244140625, 90.11000061035156, 88.43000030517578, 87.56999969482422, 83.80000305175781, 85.94000244140625, 87.44000244140625, 89.37000274658203]'

In [9]:
price.loc[price.index[-10:]]

,Date,Open,High,Low,Close,Adj Close,Volume,Change,Direction,OBV,SMA/short,SMA/long,MACD,Signal Line,RSI,SlowK,SlowD
190,2023-04-17,90.230003,90.690002,88.300003,89.870003,89.870003,47250800,-1.879997,DOWN,-732695600,93.301000,88.0070,-0.491930,0.681663,45.145154,17.534894,21.279179
191,2023-04-18,91.610001,92.160004,89.330002,89.779999,89.779999,46246300,-0.090004,DOWN,-778941900,92.623000,88.0808,-0.745269,0.396277,44.949710,14.395923,18.299982
192,2023-04-19,88.510002,90.540001,88.220001,89.940002,89.940002,37344500,0.160004,UP,-741597400,92.030000,88.2060,-0.884237,0.140174,45.402231,14.541632,15.490816
193,2023-04-20,88.830002,91.580002,88.730003,90.110001,90.110001,47082700,0.169998,UP,-694514700,91.785001,88.2900,-0.939592,-0.075779,45.910961,14.969791,14.635782
194,2023-04-21,89.800003,89.800003,88.059998,88.430000,88.430000,41067500,-1.680000,DOWN,-735582200,91.381001,88.3648,-1.141329,-0.288889,41.768910,13.004722,14.172048
195,2023-04-24,88.110001,88.639999,86.339996,87.570000,87.570000,40423800,-0.860001,DOWN,-776006000,90.590000,88.4520,-1.348177,-0.500747,39.789909,12.459332,13.477948
196,2023-04-25,86.900002,87.059998,83.760002,83.800003,83.800003,52771500,-3.769997,DOWN,-828777500,89.567001,88.4984,-1.876782,-0.775954,32.516697,9.888531,11.784195
197,2023-04-26,86.510002,87.349998,85.320000,85.940002,85.940002,47786600,2.139999,UP,-780990900,88.928001,88.5546,-1.958120,-1.012387,39.299412,10.388809,10.912224
198,2023-04-27,86.410004,87.570000,84.089996,87.440002,87.440002,53087200,1.500000,UP,-727903700,88.463001,88.5844,-1.799338,-1.169777,43.579992,12.639880,10.972406
199,2023-04-28,87.019997,89.750000,86.440002,89.370003,89.370003,50652100,1.930000,UP,-677251600,88.225002,88.6682,-1.430691,-1.221960,48.602310,20.950943,14.659877


In [10]:
balance_sheet_agent.run("What is the Equity in 2022?")



> Entering new AgentExecutor chain...
Thought: I need more information about the dataframe to answer this question.
Action: Check the columns of the dataframe.
Action Input: `print(df.columns)`
Observation: Check the columns of the dataframe. is not a valid tool, try another one.
Thought:I need to check the unique values in the 'year' column to see if there is data for 2022.
Action: Use the unique() method on the 'year' column.
Action Input: `print(df['year'].unique())`
Observation: Use the unique() method on the 'year' column. is not a valid tool, try another one.
Thought:I need to filter the dataframe to only include data for 2022.
Action: Use boolean indexing to filter the dataframe.
Action Input: `df_2022 = df[df['year'] == 2022]`
Observation: Use boolean indexing to filter the dataframe. is not a valid tool, try another one.
Thought:Now that I have a dataframe with data for 2022, I can check the Equity value.
Action: Use the loc[] method to select the row with the Equity value f

OutputParserException: Could not parse LLM output: `The final answer is that I cannot answer the question without more information about the dataframe.`

## Check the agent setup

In [ ]:
stat_agent.prep_outputs

In [ ]:
stat_agent.parse_file

In [ ]:
stat_agent.agent.llm_chain.prompt

In [ ]:
stat_agent.verbose

## Set up tools

In [14]:
from langchain import SerpAPIWrapper
from langchain.utilities import PythonREPL
from langchain.agents import Tool

In [27]:
search = SerpAPIWrapper()
python_repl = PythonREPL()
tools = [
    Tool(
        name = "Search",
        func = search.run,
        description = "useful for when you need to answer questions about current events. when I tell you 'use search', then use \
        this tool."
    ),
    Tool(
        name='Python',
        func=python_repl.run,
        description="useful for when you need to run python code to find the answer. When I tell you to 'use python', then use \
        this tool"
    ),
    Tool(
        name="Price history tool",
        func=price_agent,
        description="Useful when you need to access historical price data to answer questions about the stock price."
    ), 
]

## Set up Memory

In [24]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)

## Intialize agent

In [25]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [26]:
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=memory
    )

AttributeError: 'list' object has no attribute 'is_single_input'

In [ ]:
agent.memory

In [ ]:
agent.run("Please find the AMD's price movement and News. Please make sure the date is matched.")

In [ ]:
agent.agent.llm_chain

In [ ]:
agent.agent.llm_chain.prompt

## Work on this output parser to see if the error will fix

`agent.agent.llm_chain.prompt.output_parser`



In [ ]:
agent.agent.llm_chain.prompt.output_parser

In [ ]:
for i in range(len(agent.agent.llm_chain.prompt.messages)):
    try:
        print(f"{i+1}: {agent.agent.llm_chain.prompt.messages[i].prompt.template}")
    except:
        print(f"{i+1}: {agent.agent.llm_chain.prompt.messages[i]}")